In [7]:
import PySimpleGUI as sg
import numpy as np
import pandas as pd
from PIL import Image,ImageTk
import cv2
import io
import base64
from tabulate import tabulate
from mysql import connector

def get_img_data(f, first=False):
    img = Image.open(f)
    if first:                     
        bio = io.BytesIO()
        img.save(bio, format="PNG")
        del img
        return bio.getvalue()
    return ImageTk.BitmapImage(img)

def sqlconnection(h,r,p):
    con = connector.connect(
            host = h, 
            user = r, 
            password = p)     
    return con

def open_window(mysqlcon):
    
    button_layout = [
                    [sg.Text("       General Querying",size = (20,1), justification = "center",font=("Helvetica", 12))],
                    [sg.Text("       "),sg.Button("Show Databases",size = (15,1),font='sfprodisplay 10 bold')],
                    [sg.Text("       "),sg.Button("User Query",size = (15,1),font='sfprodisplay 10 bold')],
                    [sg.Text(" ")],
                    [sg.Text("Customer database operations",font=("Helvetica", 12))],
                    [sg.Text(" "),sg.Button("Number Of Customers",size = (20,1),font='sfprodisplay 10 bold')],
                    [sg.Text(" "),sg.Button("Cities And Offices",size = (20,1),font='sfprodisplay 10 bold')],
                    [sg.Text(" "),sg.Button("Productlines",size = (20,1),font='sfprodisplay 10 bold')],
                    [sg.Text(" "),sg.Button("Top 5 Customers",size = (20,1),font='sfprodisplay 10 bold')],
                    [sg.Text(" "),sg.Button("Customer ER Diagram",size = (20,1),font='sfprodisplay 10 bold')]
                    ]

    layout1 = [[sg.Text("Welcome to the Database Application", size = (60,1),justification = "center",font=("Helvetica", 14))],
               [sg.Text("Title:", size = (5,1),font=("Helvetica", 12)),sg.Text("",size=(50,1),font=("Helvetica", 12),key="head")],
               [sg.Text("Query:",size = (5,4),font=("Helvetica", 12)),sg.Text("",size=(70,4),key=("query"),font=("Helvetica", 12))],
               [sg.Text("Output:", size = (5,8),font=("Helvetica", 12)),sg.Text("",size = (70,8),key="new",font=("Helvetica", 12))],
               [sg.Image(size = (200,200),key="image")]]
    
    layout = [[sg.Column(button_layout),
        sg.VSeperator(),
        sg.Column(layout1)]]
    
    window = sg.Window("Customer Database", layout, modal=True)
    
    while True:
        event, values = window.read()   
        
        if event == "Exit" or event == sg.WIN_CLOSED:
            break  
                  
        cur = mysqlcon.cursor()
        
        if event == "Show Databases":
            q= "show databases;"
            cur.execute(q)
            db = []
            for i in cur:
                db.append(str(i))
            dbnames = ''
            for i in db:
                dbnames = dbnames + "   " + i[2:len(i)-3]
                
            window['head'].Update("List of the databases available")
            window['query'].Update(q)
            window['new'].Update(dbnames)
            window['image'].Update(data = None)
            
        elif event == "Number Of Customers":
            q = "select count(customerName) from customer.customers;"
            cur.execute(q)
            x=0
            for i in cur:
                x= list(i)
                
            for i in x:
                window['head'].Update("Total number of customers in the customer database")
                window['query'].Update(q)
                window['new'].Update(i)
                window['image'].Update(data = None)
        
        elif event == "Customer ER Diagram":
            file = "C:/Users/megha/Documents/woxsen/term 3/sql/project/temp.png"
            window['head'].Update("Customer ER diagram")
            window['query'].Update("")
            window['new'].Update("")
            window["image"].update(data=get_img_data(file, first=True))
        
        elif event == "User Query":
            q = sg.popup_get_text("enter query")
            cur.execute(q)
            db = []
            
            for i in cur:
                db.append(i)
            
            window['head'].Update("User Query")
            window['query'].Update(q)
            window['new'].Update(db)
            window['image'].Update(data = None)
        
        elif event == "Cities And Offices":
            q = "select o.city, count(e.officeCode) from customer.employees e inner join customer.offices o on  o.officeCode = e.officeCode group by e.officeCode;"
            cur.execute(q)
            
            db = []
            
            for i in cur:
                db.append(list(i))
                
            for x in db:
                x[0] = str(x[0])
                x[1] = str(x[1])
                
            s ="           City              Number of Offices\n"  
            s =s + "---------------------       ------------------------- \n"
            
            pd = 0
            for x in db:
                if len(x[0]) > pd:
                    pd = len(x[0]) 
        
        
            for x in db:
                s = s + x[0]
                
                if len(x[0]) < pd + 4:
                    n = pd - len(x[0])+ 5

                    
                    for i in range(0, n+2):
                             s = s + "  "
                            
                if len(x[1]) < 2:
                    s = s+ "  " + x[1] + "\n"
                else:
                    s = s+ " " + x[1] + "\n"
            
                 
            window['head'].Update("Number of offices in each city")
            window['query'].Update(q)
            window['new'].Update(s)
            window['image'].Update(data = None)
        
        elif event == "Productlines":
            q="select productLine,count(productline) from customer.products group by productLine;"
            cur.execute(q)
            
            db = []
            
            for i in cur:
                db.append(list(i))
            for x in db:
                x[0] = str(x[0])
                x[1] = str(x[1])
                
            s ="   Productline                   Count\n"  
            s =s + "-------------------------       -------------- \n"
            
            pd = 0
            for x in db:
                if len(x[0]) > pd:
                    pd = len(x[0]) 
        
        
            for x in db:
                s = s + x[0]
                
                if len(x[0]) < pd + 2:
                    n = pd - len(x[0])+3
                    
                    for i in range(0, n+2):
                             s = s + "  "
                            
                if len(x[1]) < 2:
                    s = s+ "  " + x[1] + "\n"
                else:
                    s = s+ " " + x[1] + "\n"
                                                         
                                   
            window['head'].Update("Count of each productline")
            window['query'].Update(q)
            window['new'].Update(s)
            window['image'].Update(data = None)
        
        elif event == "Top 5 Customers":
            q = "select  c.customerNumber, c.customerName, sum(p.amount) as TotalAmount from customer.customers c inner join customer.payments p on c.customerNumber = p.customerNumber group by p.customerNumber order by sum(p.amount) desc limit 5;"
            cur.execute(q)
            db = []
            
            for i in cur:
                db.append(list(i))
           
        
            s ="Customer Number               Customer Name                    Total Amount  \n"  
            s =s + "----------------------------       ------------------------------             ------------------------  \n"
            for x in db:
                s = s + "            "   + str(x[0]) + "                     " + str(x[1]) + "              " + str(x[2]) + "\n"
                 
            window['head'].Update("The top five customer orders")
            
            window['query'].Update(q)
            window['new'].Update(s)
            window['image'].Update(data = None)
            
            
            
layout = [
    [sg.Text("Please connect to the database for further details")],
    [sg.Text('',size=(5,1)),sg.Button('connect to the database',size=(20,1),font='sfprodisplay 10 bold', key = 'connect to database')]
]


win1 = sg.Window('Database',layout)


while True:
    event, values = win1.read()
     
    if event in (sg.WIN_CLOSED, 'Exit'):
        break
    if event == sg.WIN_CLOSED or event == 'Exit':
        break
    
    if event == 'connect to database':
        h = sg.popup_get_text("host")
        r = sg.popup_get_text("username")
        p = sg.popup_get_text("password", password_char="*")
        mysqlcon = sqlconnection(h,r,p)
        win1.close()
        open_window(mysqlcon)
